In [2]:
# disable the proxy
!conda install -c conda-forge skorch


CondaSSLError: Encountered an SSL error. Most likely a certificate verification issue.

Exception: HTTPSConnectionPool(host='conda.anaconda.org', port=443): Max retries exceeded with url: /conda-forge/win-64/current_repodata.json (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1129)')))




In [1]:
import numpy as np
from sklearn.datasets import make_classification
from torch import nn
from skorch import NeuralNetClassifier

X, y = make_classification(1000, 20, n_informative=10, random_state=0)
X = X.astype(np.float32)
y = y.astype(np.int64)

class MyModule(nn.Module):
    def __init__(self, num_units=10, nonlin=nn.ReLU()):
        super().__init__()

        self.dense0 = nn.Linear(20, num_units)
        self.nonlin = nonlin
        self.dropout = nn.Dropout(0.5)
        self.dense1 = nn.Linear(num_units, num_units)
        self.output = nn.Linear(num_units, 2)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = self.dropout(X)
        X = self.nonlin(self.dense1(X))
        X = self.softmax(self.output(X))
        return X



OSError: [WinError 126] 找不到指定的模块。 Error loading "D:\Anaconda\lib\site-packages\torch\lib\caffe2_nvrtc.dll" or one of its dependencies.

In [ ]:
net = NeuralNetClassifier(
    MyModule,
    max_epochs=10,
    lr=0.1,
    # Shuffle training data on each epoch
    iterator_train__shuffle=True,
)

net.fit(X, y)
y_proba = net.predict_proba(X)
print(y_proba)

不同的MKL算法有不同的优缺点，主要取决于它们的组合策略、优化目标、计算复杂度和可扩展性等方面。¹

一般来说，MKL算法可以分为两大类：基于核对齐的算法和基于边缘最大化的算法。²

基于核对齐的算法，如CKA、FHeuristic等，是一类启发式的方法，它们通过最大化核函数与目标函数之间的相似度来选择合适的核组合。这类算法的优点是计算简单，可以在闭式形式下得到解，而且不需要调节超参数。缺点是核对齐并不一定反映了分类性能，而且可能导致过拟合。²³

基于边缘最大化的算法，如EasyMKL、SimpleMKL、MEMO等，是一类基于优化理论的方法，它们通过最大化分类器的间隔或者正则化项来选择合适的核组合。这类算法的优点是可以保证分类性能和泛化能力，而且可以处理非线性和非凸的问题。缺点是计算复杂度高，需要迭代求解，而且需要调节超参数。²⁴

具体来说，不同的MKL算法之间还有一些细微的差别，例如：

- EasyMKL是一种快速且内存高效的基于边缘的组合方法，它使用了一个简单的凸优化问题，并利用了投影梯度下降法来求解。
- SimpleMKL是一种交替优化的方法，它在每次迭代中固定一个变量（核权重或者分类器参数），然后优化另一个变量。它使用了SMO算法来求解子问题，并利用了稀疏性来加速计算。
- MEMO是一种同时最大化边缘和最小化复杂度的方法，它使用了一个非凸优化问题，并利用了次梯度下降法和启发式搜索来求解。
- CKA是一种基于核对齐的方法，它使用了一个简单的二次规划问题，并利用了拉格朗日对偶性来求解。
- FHeuristic是一种基于核对齐的启发式方法，它使用了一个贪心策略来选择最佳的核组合，并利用了核函数之间的相似度来加速计算。

总之，不同的MKL算法有各自的适用场景和局限性，需要根据具体的问题和数据来选择合适的方法。

源: 与必应的对话， 2023/4/25
(1) 多核学习和深度学习的关系是什么？ - 知乎. https://www.zhihu.com/question/351929857.
(2) 矩阵运算库blas, cblas, openblas, atlas, lapack, mkl之间有什么关系，在性能上区别大吗？. https://www.zhihu.com/question/27872849.
(3) 特征提取-特征后期融合-多核学习方法（MKL） - CSDN博客. https://blog.csdn.net/wuhang19911024/article/details/46287639.
(4) 有害同义突变预测的计算方法的比较与整合 - 腾讯云开发者社区-腾讯云. https://cloud.tencent.com/developer/article/1965928.

base.py这个文件是MKLpy框架的一部分，它定义了一些基类和接口，用于实现不同的多核学习（MKL）算法。¹

具体来说，这个文件包含了以下几个类：

- Solution：这是一个用于存储MKL算法的结果的类，包括核权重、分类器参数、训练时间等属性。
- Cache：这是一个用于缓存核矩阵的类，可以提高计算效率，避免重复计算。
- MKL：这是一个抽象基类，用于定义MKL算法的通用接口，包括fit、predict、decision_function等方法。它继承了sklearn的BaseEstimator和ClassifierMixin类，因此可以兼容sklearn的API。
- OneStepMKL：这是一个继承自MKL的基类，用于实现一步式的MKL算法，即在一次优化中同时学习核权重和分类器参数。它定义了一个_combine_kernels方法，用于根据核权重计算组合核矩阵。
- TwoStepMKL：这是一个继承自MKL的基类，用于实现两步式的MKL算法，即在两次优化中分别学习核权重和分类器参数。它定义了一个initialize_optimization方法，用于初始化优化过程，以及一个do_step方法，用于执行一次迭代。

这个文件还包含了一些辅助函数，如_prepare、set_params、get_params等，用于处理数据和参数。

源: 与必应的对话， 2023/4/25
(1) MKLpy/base.py at master · IvanoLauriola/MKLpy - Github. https://github.com/IvanoLauriola/MKLpy/blob/master/MKLpy/algorithms/base.py.
(2) MKLpy/multiclass.py at master · IvanoLauriola/MKLpy · GitHub. https://github.com/IvanoLauriola/MKLpy/blob/master/MKLpy/multiclass.py.
(3) MKLpy/examples_training.md at master · IvanoLauriola/MKLpy - Github. https://github.com/IvanoLauriola/MKLpy/blob/master/docs/examples/examples_training.md.

OneStepMKL和TwoStepMKL是两种不同的MKL算法的基类，它们分别实现了一步式和两步式的优化策略。¹²

一步式的优化策略，即在一次优化中同时学习核权重和分类器参数。这种方法的优点是计算简单，可以在闭式形式下得到解，而且不需要迭代求解。缺点是可能导致过拟合，而且对于非凸的问题可能陷入局部最优。¹³

OneStepMKL类继承了MKL类，并重写了_combine_kernels方法，用于根据核权重计算组合核矩阵。具体的算法实现过程如下：

- 初始化一个Solution对象，用于存储优化结果。
- 调用_prepare方法，用于检查输入数据和参数，并进行标准化处理。
- 调用_fit方法，用于根据具体的算法实现来计算核权重和分类器参数，并将结果存储在Solution对象中。
- 调用_combine_kernels方法，用于根据核权重和核矩阵列表来计算组合核矩阵，并将结果存储在Solution对象中。
- 返回Solution对象。

两步式的优化策略，即在两次优化中分别学习核权重和分类器参数。这种方法的优点是可以保证分类性能和泛化能力，而且可以处理非线性和非凸的问题。缺点是计算复杂度高，需要迭代求解，而且需要调节超参数。¹ 

TwoStepMKL类继承了MKL类，并重写了initialize_optimization和do_step方法，用于实现交替优化的过程。具体的算法实现过程如下：

- 初始化一个Solution对象，用于存储优化结果。
- 调用_prepare方法，用于检查输入数据和参数，并进行标准化处理。
- 调用initialize_optimization方法，用于初始化优化过程，包括设置初始核权重、学习率、调度器、回调函数等。
- 进入一个循环，直到达到最大迭代次数或者收敛条件。
  - 调用do_step方法，用于执行一次迭代，包括更新核权重、更新分类器参数、更新学习率等，并将结果存储在Solution对象中。
  - 调用_combine_kernels方法，用于根据核权重和核矩阵列表来计算组合核矩阵，并将结果存储在Solution对象中。
  - 调用回调函数，用于监控优化过程或者执行其他操作。
- 返回Solution对象。

两者的主要差异在于优化策略、计算复杂度、泛化能力等方面。¹²更新核权重是指在多核学习（MKL）算法中，根据损失函数和优化方法，调整不同核函数的组合系数，从而得到一个更好的组合核矩阵。¹²

跟神经网络的更新权重有一些相似之处，但也有一些区别。³ 

相似之处在于，都是基于梯度下降的原理，通过计算损失函数对权重的偏导数，来确定更新的方向和幅度。都可以使用一些优化技巧，如学习率衰减、动量法、正则化等，来加速收敛或防止过拟合。

区别在于，MKL算法中的核权重是对应于不同的核函数，而神经网络中的权重是对应于不同的神经元连接。MKL算法中的核权重通常是非负且和为1的，而神经网络中的权重没有这样的限制。MKL算法中的核权重可以用一个向量表示，而神经网络中的权重通常是一个矩阵或张量。MKL算法中的核权重只影响组合核矩阵的计算，而神经网络中的权重会影响每一层的输出和激活函数。MKL算法中的核权重可以用一步式或两步式的方法来更新，而神经网络中的权重通常用反向传播的方法来更新。

源: 与必应的对话， 2023/4/25
(1) CNN卷积神经网络学习过程（权值更新）_卷积神经网络如何更新权重_LifeBackwards的博客-CSDN博客. https://blog.csdn.net/daodaipsrensheng/article/details/119346487.
(2) 神经网络权重更新的原理和方法 - CSDN博客. https://blog.csdn.net/liumingchun13/article/details/104988292.
(3) 卷积神经网络（CNN）的训练过程、权值更新_卷积神经网络权重更新_boringApp的博客-CSDN博客. https://blog.csdn.net/boringApp/article/details/127040190.

源: 与必应的对话， 2023/4/25
(1) Training - MKLpy - Read the Docs. https://mklpy.readthedocs.io/en/latest/examples/examples_training/.
(2) MKL - MKLpy - Read the Docs. https://mklpy.readthedocs.io/en/latest/API/MKL/.
(3) MKLpy/examples_training.md at master · IvanoLauriola/MKLpy. https://github.com/IvanoLauriola/MKLpy/blob/master/docs/examples/examples_training.md.

In [5]:
import numpy as np
K = np.array([[1,6],[2,2]]) # 用你的矩阵数据替换
D = np.diag(K) # 提取K的对角线元素
M = D * D[:, None] # 将D与它的转置相乘，得到一个对角矩阵
K / M # 用M的每个元素去除K的每一列

array([[1. , 3. ],
       [1. , 0.5]])

这个程序是一个实现了**中心核对齐（CKA）**算法的类，它继承了**OneStepMKL**类，用于在一次优化中同时学习核权重和分类器参数。¹

程序的主要结构如下：

- 导入所需的库和模块，如numpy、sklearn、MKLpy等。
- 定义一个**CKA**类，继承自**OneStepMKL**类。
- 在类的初始化方法中，设置一些参数，如正则化系数、最大迭代次数、收敛阈值等，并调用父类的初始化方法。
- 重写**_fit**方法，用于根据CKA算法的具体实现来计算核权重和分类器参数。具体步骤如下：
  - 初始化核权重为均匀分布，即每个核函数的权重相等。
  - 初始化一个缓存对象，用于存储和加速核矩阵的计算。
  - 进入一个循环，直到达到最大迭代次数或者收敛条件。
    - 计算组合核矩阵，即根据核权重和核矩阵列表来计算加权和。
    - 计算中心化后的组合核矩阵，即根据公式 $$K_c = K - \frac{1}{n}K\mathbf{1} - \frac{1}{n}\mathbf{1}K + \frac{1}{n^2}\mathbf{1}K\mathbf{1}$$ 来去除数据的均值影响。
    - 训练一个线性SVM分类器，使用中心化后的组合核矩阵作为输入，并获取分类器的参数和支持向量。
    - 计算每个核函数与组合核函数的中心核对齐系数，即根据公式 $$\text{CKA}(K_i,K) = \frac{\langle K_i, K \rangle_F}{\sqrt{\langle K_i, K_i \rangle_F \langle K, K \rangle_F}}$$ 来衡量两个核函数之间的相似度。其中 $$\langle A, B \rangle_F$$ 表示两个矩阵的Frobenius内积，即 $$\langle A, B \rangle_F = \sum_{i,j} A_{ij} B_{ij}$$
    - 更新核权重，即根据每个核函数与组合核函数的中心核对齐系数来分配新的权重，并归一化使得和为1。
    - 检查收敛条件，即根据新旧核权重之间的差异来判断是否达到稳定状态。
  - 返回核权重和分类器参数。
- 重写**_combine_kernels**方法，用于根据核权重和缓存对象来计算组合核矩阵，并返回结果。

这个程序是如何使用OneStepMKL特性的呢？主要体现在以下几个方面：

- 它继承了OneStepMKL类，从而可以复用其父类的一些属性和方法，如初始化方法、预处理方法、预测方法等。
- 它只需要重写_fit方法和_combine_kernels方法，来实现自己的优化策略和组合方式，而不需要关心其他细节。
- 它可以兼容sklearn的API，从而可以方便地使用sklearn提供的一些工具和功能，如交叉验证、网格搜索、评估指标等。²

源: 与必应的对话， 2023/4/25
(1) MKLpy/multiclass.py at master · IvanoLauriola/MKLpy · GitHub. https://github.com/IvanoLauriola/MKLpy/blob/master/MKLpy/multiclass.py.
(2) MKLpy/examples_training.md at master · IvanoLauriola/MKLpy - Github. https://github.com/IvanoLauriola/MKLpy/blob/master/docs/examples/examples_training.md.
(3) MKLpy/CKA.py at master · IvanoLauriola/MKLpy - Github. https://github.com/IvanoLauriola/MKLpy/blob/master/MKLpy/algorithms/CKA.py.

SVM算法中，最优的α向量是指满足对偶问题的拉格朗日乘子向量，它可以用以下公式表示：
αmax​i=1∑m​αi​−21​i=1∑m​j=1∑m​yi​yj​αi​αj​(xi​⋅xj​)s.t.i=1∑m​yi​αi​=00≤αi​≤C,i=1,2,...,m
其中，m是样本的个数，yi​是样本的标签（+1或-1），xi​是样本的特征向量，C是惩罚参数，控制了软间隔的大小。对偶问题的目标是最大化α向量的值，同时满足约束条件。一般可以使用序列最小优化（SMO）算法来求解这个问题。

这个程序是一个实现了**广义正则化对齐（GRAM）**算法的类，它继承了**TwoStepMKL**类，用于在两次优化中分别学习核权重和分类器参数。¹

程序的主要结构如下：

- 导入所需的库和模块，如numpy、sklearn、MKLpy等。
- 定义一个**GRAM**类，继承自**TwoStepMKL**类。
- 在类的初始化方法中，设置一些参数，如正则化系数、最大迭代次数、收敛阈值、学习率等，并调用父类的初始化方法。
- 重写**initialize_optimization**方法，用于初始化优化过程，包括设置初始核权重、学习率、调度器等。
- 重写**do_step**方法，用于执行一次迭代，包括更新核权重、更新分类器参数、更新学习率等。具体步骤如下：
  - 计算组合核矩阵，即根据核权重和核矩阵列表来计算加权和。
  - 计算中心化后的组合核矩阵，即根据公式 $$K_c = K - \frac{1}{n}K\mathbf{1} - \frac{1}{n}\mathbf{1}K + \frac{1}{n^2}\mathbf{1}K\mathbf{1}$$ 来去除数据的均值影响。
  - 训练一个线性SVM分类器，使用中心化后的组合核矩阵作为输入，并获取分类器的参数和支持向量。
  - 计算每个核函数与组合核函数的广义正则化对齐系数，即根据公式 $$\text{GRA}(K_i,K) = \frac{\langle K_i, K \rangle_F}{\sqrt{\langle K_i, K_i \rangle_F + \lambda \langle K, K \rangle_F}}$$ 来衡量两个核函数之间的相似度。其中 $$\langle A, B \rangle_F$$ 表示两个矩阵的Frobenius内积，即 $$\langle A, B \rangle_F = \sum_{i,j} A_{ij} B_{ij}$$ ，而 $\lambda$ 是一个正则化系数，用于控制对齐系数的大小。
  - 更新核权重，即根据每个核函数与组合核函数的广义正则化对齐系数来分配新的权重，并归一化使得和为1。
  - 更新学习率，即根据一个指数衰减的调度器来降低学习率。

这个程序是如何使用TwoStepMKL特性的呢？主要体现在以下几个方面：

- 它继承了TwoStepMKL类，从而可以复用其父类的一些属性和方法，如初始化方法、预处理方法、预测方法等。
- 它只需要重写initialize_optimization方法和do_step方法，来实现自己的优化策略和组合方式，而不需要关心其他细节。
- 它可以兼容sklearn的API，从而可以方便地使用sklearn提供的一些工具和功能，如交叉验证、网格搜索、评估指标等。²

它与CKA有何不同呢？主要体现在以下几个方面：

- 它使用了两步式的优化策略，而CKA使用了一步式的优化策略。这意味着它需要迭代求解，而CKA可以在闭式
- 它使用了广义正则化对齐系数，而CKA使用了中心核对齐系数。这意味着它引入了一个正则化项，用于控制对齐系数的大小，而CKA没有这样的项。
- 它使用了一个指数衰减的调度器，而CKA没有使用学习率。这意味着它可以动态地调整学习速度，而CKA使用固定的步长。

源: 与必应的对话， 2023/4/25
(1) MKLpy/multiclass.py at master · IvanoLauriola/MKLpy · GitHub. https://github.com/IvanoLauriola/MKLpy/blob/master/MKLpy/multiclass.py.
(2) MKLpy/model_selection.py at master · IvanoLauriola/MKLpy - Github. https://github.com/IvanoLauriola/MKLpy/blob/master/MKLpy/model_selection.py.
(3) MKLpy/GRAM.py at master · IvanoLauriola/MKLpy - Github. https://github.com/IvanoLauriola/MKLpy/blob/master/MKLpy/algorithms/GRAM.py.

正则化是一种用于防止模型过拟合的技术，它在损失函数中加入一个正则项，用于惩罚模型的复杂度。¹

正则化的系数是一个用于调节正则项和损失项之间平衡的参数，它通常是一个非负的实数。²

正则化的系数的含义是：

- 如果正则化的系数为0，那么正则项就没有作用，相当于没有使用正则化。
- 如果正则化的系数很大，那么正则项就会占据主导地位，导致模型过于简单，可能出现欠拟合。
- 如果正则化的系数适中，那么正则项和损失项就会达到一个平衡点，使得模型既不过拟合也不欠拟合。³

因此，选择合适的正则化的系数是很重要的，一般可以通过交叉验证或网格搜索等方法来寻找最优的值。

源: 与必应的对话， 2023/4/25
(1) ML 入门：归一化、标准化和正则化 - 知乎 - 知乎专栏. https://zhuanlan.zhihu.com/p/29957294.
(2) 机器学习必知必会：正则化 - 知乎 - 知乎专栏. https://zhuanlan.zhihu.com/p/67931198.
(3) 一篇文章完全搞懂正则化（Regularization） - CSDN博客. https://blog.csdn.net/weixin_41960890/article/details/104891561.

Hinge Loss是一种常用于支持向量机（SVM）的损失函数，它的数学表达式为：$L(y)=max(0,1-yf(x))$，其中y是样本的真实标签（+1或-1），$f(x)$是模型的预测输出（$w^Tx+b$）。hinge loss的含义是，如果样本被正确分类且函数间隔大于等于1，那么损失为0；否则损失为$1-yf(x)$。

hinge loss的梯度传播是指根据损失函数对模型参数w和b求偏导数，从而更新参数的过程。hinge loss的梯度传播可以根据以下公式进行：

如果$yf(x)\geq 1$，那么$L(y)=0$，对w和b求偏导数都为0，即$$\frac {\partial L} {\partial w}=0，\frac {\partial L} {\partial b}=0$$
这意味着参数不需要更新。
如果$yf(x)< 1$，那么$L(y)=1-yf(x)$，对w和b求偏导数分别为$$\frac {\partial L} {\partial w}=-yx和\frac {\partial L} {\partial b}=-y$$
这意味着参数需要按照梯度下降的方向更新。

In [13]:
import torch
def rbf_kernel(tensor,sig=1):
    n=tensor.shape[1]
    K=tensor/sig^2
    d=torch.diag(K)
    K=K-torch.ones((n, 1))*d.t()/2-d*torch.ones((1, n))/2
    print(K)
    return torch.exp(K)
rbf_kernel(torch.tensor([[1.0,3.0],[2.0,4.0]]),sig=1)

OSError: [WinError 126] 找不到指定的模块。 Error loading "D:\Anaconda\lib\site-packages\torch\lib\caffe2_nvrtc.dll" or one of its dependencies.

将每个元素减去对应点到自己的距离的一半，是一种对RBF核函数的变形，它可以使得核函数的值在[0, 1]之间，而不是在[0, +∞)之间。这样做的目的是为了使得核函数的值更加稳定，避免数值溢出或者欠拟合的问题¹²。RBF核函数是一种常用的核函数，它可以用来度量两个样本之间的相似度，它的定义是：

$$k(x, x') = \exp(-\gamma \|x - x'\|^2)$$

其中，$\gamma$是一个超参数，控制了核函数的宽度。如果将每个元素减去对应点到自己的距离的一半，那么核函数就变成了：

$$k(x, x') = \exp(-\gamma (\|x - x'\|^2 - \frac{1}{2}(\|x\|^2 + \|x'\|^2)))$$

这样，当$x = x'$时，核函数的值为1；当$x$和$x'$越来越远时，核函数的值趋近于0。

源: 与必应的对话， 2023/4/21
(1) Radial basis function kernel - Wikipedia. https://en.wikipedia.org/wiki/Radial_basis_function_kernel.
(2) sklearn.gaussian_process.kernels .RBF - scikit-learn. https://scikit-learn.org/stable/modules/generated/sklearn.gaussian_process.kernels.RBF.html.
(3) 核函数&径向基核函数 (Radial Basis Function)--RBF - CSDN博客. https://blog.csdn.net/huang1024rui/article/details/51510611.

sklearn中rbf，sigmoid，poly kernel是一些常用的核函数，它们可以用来在支持向量机或者其他核方法中实现非线性的映射。它们的用法是：

- rbf核函数是径向基核函数，也叫高斯核函数，它的形式是$k(x, x') = \exp(-\gamma \|x - x'\|^2)$，其中$\gamma$是一个超参数，控制了核函数的宽度。它可以用来度量两个样本之间的相似度，它的值在[0, 1]之间。要使用rbf核函数，可以在sklearn中设置kernel='rbf'，并指定gamma的值¹²。
- sigmoid核函数是一个类似于sigmoid激活函数的核函数，它的形式是$k(x, x') = \tanh(\gamma x^T x' + r)$，其中$\gamma$和$r$是两个超参数，控制了核函数的形状。它可以用来实现神经网络中的多层感知机模型。要使用sigmoid核函数，可以在sklearn中设置kernel='sigmoid'，并指定gamma和coef0（对应于r）的值¹³。
- poly核函数是多项式核函数，它的形式是$k(x, x') = (\gamma x^T x' + r)^d$，其中$\gamma$，$r$和$d$是三个超参数，分别控制了核函数的缩放、偏移和次数。它可以用来实现多项式特征的映射。要使用poly核函数，可以在sklearn中设置kernel='poly'，并指定gamma，coef0（对应于r）和degree（对应于d）的值¹ 。

源: 与必应的对话， 2023/4/21
(1) sklearn.kernel_ridge - scikit-learn 1.1.1 documentation. https://scikit-learn.org/stable/modules/generated/sklearn.kernel_ridge.KernelRidge.html.
(2) sklearn.gaussian_process.kernels .RBF - scikit-learn. https://scikit-learn.org/stable/modules/generated/sklearn.gaussian_process.kernels.RBF.html.
(3) sklearn.svm.SVC — scikit-learn 1.2.2 documentation. https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html.